In [1]:
# Sauvegarde du fichier data dans l'espace de stockage Azure
from flask import Flask, render_template, request
from tensorflow.keras.models import load_model

from azureml.core import Workspace
from azureml.core.model import Model

from matplotlib.pyplot import imsave
from PIL import Image
from numpy import array
import requests

app = Flask(__name__)

# Connection à l'espace de travail d'Azure
ws = Workspace(subscription_id="d5bb9744-4790-446f-b7e1-591e22995cc7",
           resource_group="OpenClassrooms",
           workspace_name="OC_IA")
model = load_model(Model.get_model_path('Model_vgg_unet', _workspace=ws))

def load_img_from_azure(name):
    # Connection à l'espace de travail d'Azure
    url = f'https://ocia0932039034.blob.core.windows.net/azureml-blobstore-f8554f92-a33d-430c-a1ff-4d9a166c55fc/UI/data/{name}_leftImg8bit.png'
    X =  array(Image.open(requests.get(url, stream=True).raw))
    imsave('./static/origine.png',X)
    return X

@app.route('/', methods=['GET'])
def california_index():
    return render_template("index.html")


@app.route('/predict/', methods=['POST'])
def result():    
    if request.method == 'POST':
        name_img = request.form['name_img']
        X = array([load_img_from_azure(name_img)])
        y = array([[i.argmax() for i in u] for u in model.predict(X[:])])
        imsave('./static/pred.png',y.reshape((512,1024)))
    return render_template("prediction.html", name_img=name_img)

if __name__ == '__main__':
    app.run()

Metal device set to: Apple M1 Max


2022-03-08 18:04:43.766343: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-08 18:04:43.766469: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Mar/2022 18:04:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 18:04:47] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Mar/2022 18:04:47] "GET /static/css/jumbotron-narrow.css HTTP/1.1" 304 -
2022-03-08 18:04:51.347660: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-08 18:04:51.415225: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
127.0.0.1 - - [08/Mar/2022 18:04:52] "POST /predict/ HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 18:04:52] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Mar/2022 18:04:52] "GET /static/css/jumbotron-narrow.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Mar/2022 18:04:52] "GET /static/origine.png HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2022 18:04:52] "GET /static/pred.png HTTP/1.1" 200 -
